In [33]:
from csv import reader
import numpy as np
from unidecode import unidecode
import pandas as pd
import statsmodels.api as sm
import scipy as ss

In [2]:
from limpeza import main as limpa
from cria_legislatura import main as legislatura
from build import main as build
from gml_para_rede import main as rede
from conta_frentes import main as frentes

limpa()
legislatura()
build()
rede()
frentes()

----------------------
0.08808553971486761
----------------------


#### Nesse teste de hipótese estariamos avaliando qual a influência das mulheres nas votações, será que os homens possuem além de maior representativade maior facilidade para aprovação de propostas?

In [14]:
deputados = pd.read_csv('ArquivosLimpos/deputados-2015-2018.csv').dropna()

In [15]:
deputados.dtypes

deputado_id                 int64
nome                       object
idLegislaturaInicial        int64
idLegislaturaFinal          int64
siglaSexo                  object
betweenness_centrality    float64
degree_centrality         float64
eigenvecto_centrality     float64
Frentes                     int64
dtype: object

In [16]:
values = deputados['siglaSexo'].unique()

for value in values[:-1]:
    deputados[value] = [1 if v == value else 0 for v in deputados['siglaSexo']]

In [17]:
deputados['F'] = deputados['siglaSexo'].apply(lambda value: 1 if value == "F" else 0)

In [25]:
deputados

,deputado_id,nome,idLegislaturaInicial,idLegislaturaFinal,siglaSexo,betweenness_centrality,degree_centrality,eigenvecto_centrality,Frentes,M,F
0,4930,SILVIO COSTA,53,55,M,0.0,0.090610,4.965024e-11,63,1,0
1,4931,IZALCI LUCAS,53,55,M,0.0,0.090610,4.965024e-11,195,1,0
2,62881,Danilo Forte,54,56,M,0.0,0.087315,4.965024e-11,227,1,0
3,64960,MARIA HELENA,52,55,F,0.0,0.080725,4.965024e-11,99,0,1
4,66179,Norma Ayub,55,56,F,0.0,0.046129,4.965024e-11,8,0,1
...,...,...,...,...,...,...,...,...,...,...,...
603,200153,PROF. GEDEAO AMORIM,55,55,M,0.0,0.014827,4.965024e-11,6,1,0
604,202063,OSVALDO MAFRA,55,55,M,0.0,0.011532,4.965024e-11,2,1,0
605,202915,EURICELIA CARDOSO,55,55,F,0.0,0.006590,4.965024e-11,0,0,1
606,204011,PROFESSOR PACCO,55,55,M,0.0,0.006590,4.965024e-11,0,1,0


#### Influênca sexo na influência de um deputado

In [26]:
model = sm.Logit(deputados['eigenvecto_centrality'], deputados[values[:-1]])
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.085200
         Iterations 8


c:\users\duda\appdata\local\programs\python\python37-32\lib\site-packages\statsmodels\discrete\discrete_model.py:3489: RuntimeWarning:

divide by zero encountered in double_scalars



<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                            
=================================================================================
Dep. Variable:     eigenvecto_centrality   No. Observations:                  608
Model:                             Logit   Df Residuals:                      607
Method:                              MLE   Df Model:                            0
Date:                   Fri, 20 Nov 2020   Pseudo R-squ.:                     inf
Time:                           17:31:06   Log-Likelihood:                -51.801
converged:                          True   LL-Null:                        0.0000
Covariance Type:               nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
M             -4.3015      0.373    -11.543      0.000      -5.032      -3.571
==============================================================================
"""

#### Influência eficiência na inflência de um deputado

In [27]:
model = sm.OLS(deputados['eigenvecto_centrality'], deputados[['Frentes']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                  
==========================================================================================
Dep. Variable:     eigenvecto_centrality   R-squared (uncentered):                   0.096
Model:                               OLS   Adj. R-squared (uncentered):              0.094
Method:                    Least Squares   F-statistic:                              64.29
Date:                   Fri, 20 Nov 2020   Prob (F-statistic):                    5.54e-15
Time:                           17:31:10   Log-Likelihood:                          1116.6
No. Observations:                    608   AIC:                                     -2231.
Df Residuals:                        607   BIC:                                     -2227.
Df Model:                              1                                                  
Covariance Type:               nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Frentes      8.58e-05   1.07e-05      8.018      0.000    6.48e-05       0.000
==============================================================================
Omnibus:                      337.043   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1597.187
Skew:                           2.651   Prob(JB):                         0.00
Kurtosis:                       8.911   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
cor = np.corrcoef(deputados['eigenvecto_centrality'],deputados['degree_centrality'])

In [29]:
cor

array([[1.        , 0.98913504],
       [0.98913504, 1.        ]])

In [30]:
H = deputados.query('siglaSexo == "M"')['eigenvecto_centrality']
M = deputados.query('siglaSexo == "F"')['eigenvecto_centrality']

In [31]:
deputados.groupby('siglaSexo').describe()

deputado_id                                                  \
                count           mean           std      min       25%   
siglaSexo                                                               
F                62.0  147608.241935  46170.719957  64960.0  98839.25   
M               546.0  139009.498168  44734.778919   4930.0  74707.00   

                                         idLegislaturaInicial             ...  \
                50%        75%       max                count       mean  ...   
siglaSexo                                                                 ...   
F          175225.5  178945.75  202915.0                 62.0  53.919355  ...   
M          160535.5  178881.75  204760.0                546.0  53.161172  ...   

             M           F                                     
           75%  max  count mean  std  min  25%  50%  75%  max  
siglaSexo                                                      
F          0.0  0.0   62.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  
M          1.0  1.0  546.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[2 rows x 72 columns]

In [34]:
res = ss.stats.ttest_ind(H, M, equal_var=True)

In [35]:
res

Ttest_indResult(statistic=-0.7644941603797848, pvalue=0.44487039484630153)

In [38]:
ttest,pval = ss.stats.ttest_rel(H, M)
print("p-value",pval)

AttributeError: module 'statsmodels.api' has no attribute 'ttest_rel'